In [8]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -U scikit-learn

In [9]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression



# Preparing the TRAIN set

In [10]:
ruta = "/content/drive/MyDrive/data_Federico_Ortega/TRAIN.csv"
data_train = pd.read_csv(ruta)
data_train

,tokens,ner_tags
0,EudraCT Nº : 2020-005522-28,O O O O
1,Título público : Estudio para evaluar la efica...,O O O O O O O O O O O O O B-CW O O B-CW O B-CW...
2,Título científico : Estudio de fase 1-3 para e...,O O O B-CW I-CW I-CW I-CW O O O O O O O O O B-...
3,Indicación científica : Esclerosis lateral ami...,B-CW O O B-CW I-CW I-CW O B-CW O B-CW I-CW I-C...
4,Indicación pública : Esclerosis lateral amiotr...,B-CW O O B-CW I-CW I-CW
...,...,...
2303,Después de recibir la información sobre la int...,O O O O O O O O O O O O O O O O O O B-CW O B-C...
2304,EL MEDICO QUE INFORMA :,O O O O O
2305,EL PACIENTE :,O O O
2306,SU REPRESENTANTE :,O O O


In [11]:
print(data_train.iloc[3]['ner_tags'].split())
print(len(data_train.iloc[3]['ner_tags']))
type(data_train.iloc[3]['ner_tags'])

['B-CW', 'O', 'O', 'B-CW', 'I-CW', 'I-CW', 'O', 'B-CW', 'O', 'B-CW', 'I-CW', 'I-CW', 'I-CW']
52


str

In [12]:
# Testing to understand the shape of the data
lista=data_train.iloc[2]['ner_tags'].split()
len(lista)

42

In [13]:
# Testing to understand the shape of the data
text=data_train.iloc[2]['tokens'].split()
print(len(text))


42


In [14]:
data_train

,tokens,ner_tags
0,EudraCT Nº : 2020-005522-28,O O O O
1,Título público : Estudio para evaluar la efica...,O O O O O O O O O O O O O B-CW O O B-CW O B-CW...
2,Título científico : Estudio de fase 1-3 para e...,O O O B-CW I-CW I-CW I-CW O O O O O O O O O B-...
3,Indicación científica : Esclerosis lateral ami...,B-CW O O B-CW I-CW I-CW O B-CW O B-CW I-CW I-C...
4,Indicación pública : Esclerosis lateral amiotr...,B-CW O O B-CW I-CW I-CW
...,...,...
2303,Después de recibir la información sobre la int...,O O O O O O O O O O O O O O O O O O B-CW O B-C...
2304,EL MEDICO QUE INFORMA :,O O O O O
2305,EL PACIENTE :,O O O
2306,SU REPRESENTANTE :,O O O


In [15]:
# array tags contains the tags of the complex words

array_tags=data_train.loc[:,"ner_tags"].values
len(array_tags)
all_tags=[]
for tag in array_tags:
  lista_tag=tag.split()
  all_tags.append(lista_tag)

In [16]:
print(len(all_tags))
print(len(all_tags[2]))

2308
42


#### **Creating a list of tuples, each sentence with its words and tags. Each sentence is repeated for evey word and tag**

### Adding POS Tag to each word (this takes some time)

In [ ]:
pip install spacy

In [13]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 30.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
import spacy
nlp_es = spacy.load("es_core_news_sm")


# Adding POS from spaCy. This takes some time.

In [18]:
# array tokens contains the tokens of the complex words
array_tokens=data_train.loc[:,"tokens"].values
print(len(array_tokens))
all_tokens=[]
all_tags=[]
all_pos=[]
tuple_sent_token=()
list_tuple_sent_token=[]
for token, tag in zip(array_tokens, array_tags):
  lista_pos=[]
  lista_token=token.split()
  lista_tag=tag.split()
  for t in lista_token:
    pos=nlp_es(t)
    pos=pos[0].pos_
    lista_pos.append(pos)
  all_tokens.append(lista_token)
  all_tags.append(lista_tag)
  all_pos.append(lista_pos)
  tuple_sent_token=(token, lista_token, lista_pos, lista_tag)
  list_tuple_sent_token.append(tuple_sent_token)



2308


In [19]:
list_tuple_sent_token[0]

('EudraCT Nº : 2020-005522-28',
 ['EudraCT', 'Nº', ':', '2020-005522-28'],
 ['NOUN', 'DET', 'PUNCT', 'NUM'],
 ['O', 'O', 'O', 'O'])

In [20]:
list_sent_word=[]
tuple_sent=()
for element in list_tuple_sent_token:
  for word, pos, tag in zip(element[1],element[2], element[3]):
    tuple_sent=(element[0], word, pos, tag)

    list_sent_word.append(tuple_sent)


list_sent_word[0]

('EudraCT Nº : 2020-005522-28', 'EudraCT', 'NOUN', 'O')

In [21]:
df_complete = pd.DataFrame(list_sent_word, columns =['Sentence', 'Word', 'POS', 'Tag'])
df_complete

,Sentence,Word,POS,Tag
0,EudraCT Nº : 2020-005522-28,EudraCT,NOUN,O
1,EudraCT Nº : 2020-005522-28,Nº,DET,O
2,EudraCT Nº : 2020-005522-28,:,PUNCT,O
3,EudraCT Nº : 2020-005522-28,2020-005522-28,NUM,O
4,Título público : Estudio para evaluar la efica...,Título,PROPN,O
...,...,...,...,...
38686,SU REPRESENTANTE :,SU,DET,O
38687,SU REPRESENTANTE :,REPRESENTANTE,PROPN,O
38688,SU REPRESENTANTE :,:,PUNCT,O
38689,FECHA :,FECHA,PROPN,O


In [22]:
df_complete = df_complete.replace([np.nan, -np.inf], 0)


#**Generating a set of features for each word**
Example from [Kaggle](https://www.kaggle.com/code/tayyabnasir22/ner-with-logisticregression)

In [23]:
def GenerateFeaturesForSentence(word, prevWord, nextWord, pos, label):
    # A single feature per word
    return {
        "lowercase": word.lower(),
        "prevword": prevWord,
        "nextword": nextWord,
        "iscaps": str(word.isupper()),
        "istitlecase": str(word.istitle()),
        "isdigit": str(word.isdigit()),
        "pos": pos,
       }, label

examples = []
for index, group in df_complete.groupby('Sentence'):
    words = list(group['Word'])
    pos = list(group['POS'])
    tags = list(group['Tag'])

    for index, word in enumerate(words):
        if index == 0:
            prevWord = '<start>'
        else:
            prevWord = words[index - 1]

        if index + 1 < len(words):
            nextWord = words[index + 1]
        else:
            nextWord = '<end>'
        examples.append(GenerateFeaturesForSentence(word, prevWord, nextWord, pos[index], tags[index]))


In [24]:
print(len(examples))
print(examples[100])

38691
({'lowercase': 'cirrosis', 'prevword': 'de', 'nextword': 'con', 'iscaps': 'False', 'istitlecase': 'False', 'isdigit': 'False', 'pos': 'NOUN'}, 'B-CW')


This [link](https://saturncloud.io/blog/how-to-create-new-values-in-a-pandas-dataframe-column-based-on-values-from-another-column/#:~:text=To%20create%20the%20new%20column,and%20returns%20the%20corresponding%20category.) is interesting to add columns based on conditions in a dataframe.

# Processing data in the DEV set

In [25]:
ruta_dev = "/content/drive/MyDrive/data_Federico_Ortega/DEV.csv"
data_dev = pd.read_csv(ruta_dev)
data_dev

,tokens,ner_tags
0,EudraCT Nº : 2018-001167-23,O O O O
1,Título público : Un estudio de Sacituzumab Gov...,O O O O O O B-CW I-CW O O O B-CW I-CW B-CW O O...
2,Título científico : Estudio en fase II abierto...,O O O B-CW I-CW I-CW I-CW I-CW O B-CW I-CW O O...
3,Indicación pública : Cáncer urotelial maligno ...,B-CW O O B-CW I-CW B-CW O B-CW O O O B-CW I-CW...
4,Indicación científica : Carcinoma urotelial de...,B-CW O O B-CW I-CW O B-CW I-CW O B-CW I-CW B-C...
...,...,...
1709,NO,O
1710,Nombre del médico : _ _ _ _ _ _ _ _ _ _ _ _ _ ...,O O O O O O O O O O O O O O O O O O O O O O O ...
1711,Firma _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,O O O O O O O O O O O O O O O O O O O O O O O ...
1712,Fecha :,O O


## Processing the dataframe to get the words and the tags

In [26]:
array_tags_dev=data_dev.loc[:,"ner_tags"].values
len(array_tags_dev)
all_tags_dev=[]
for tag_dev in array_tags_dev:
  lista_tag_dev=tag_dev.split()
  all_tags_dev.append(lista_tag_dev)


In [27]:
print(len(all_tags_dev))
print(len(all_tags_dev[2]))

1714
20


In [28]:
array_tokens_dev=data_dev.loc[:,"tokens"].values
len(array_tokens_dev)
all_tokens_dev=[]
for token_dev in array_tokens_dev:
  lista_token_dev=token_dev.split()
  all_tokens_dev.append(lista_token_dev)

In [29]:
print(len(all_tokens_dev))
print(len(all_tokens_dev[2]))

1714
20


# Adding POS from spaCy to the DEV set.

In [30]:
# array tokens contains the tokens of the complex words
array_tokens_dev=data_dev.loc[:,"tokens"].values
print(len(array_tokens_dev))
all_tokens_dev=[]
all_tags_dev=[]
all_pos_dev=[]
tuple_sent_token_dev=()
list_tuple_sent_token_dev=[]
for token_dev, tag_dev in zip(array_tokens_dev, array_tags_dev):
  lista_pos_dev=[]
  lista_token_dev=token_dev.split()
  lista_tag_dev=tag_dev.split()
  for t_dev in lista_token_dev:

    pos_dev=nlp_es(t_dev)
    pos_dev=pos_dev[0].pos_
    lista_pos_dev.append(pos_dev)
  all_tokens_dev.append(lista_token_dev)
  all_tags_dev.append(lista_tag_dev)
  all_pos_dev.append(lista_pos_dev)
  tuple_sent_token_dev=(token_dev, lista_token_dev, lista_pos_dev, lista_tag_dev)
  list_tuple_sent_token_dev.append(tuple_sent_token_dev)



1714


In [31]:
list_sent_word_dev=[]
tuple_sent_dev=()
for element_d in list_tuple_sent_token_dev:
  for word_d, pos_d, tag_d in zip(element_d[1],element_d[2], element_d[3]):
    tuple_sent_dev=(element_d[0], word_d, pos_d, tag_d)

    list_sent_word_dev.append(tuple_sent_dev)

list_sent_word_dev[0]

('EudraCT Nº : 2018-001167-23', 'EudraCT', 'NOUN', 'O')

In [32]:
df_dev_complete = pd.DataFrame(list_sent_word_dev, columns =['Sentence', 'Word', 'POS', 'Tag'])
df_dev_complete

,Sentence,Word,POS,Tag
0,EudraCT Nº : 2018-001167-23,EudraCT,NOUN,O
1,EudraCT Nº : 2018-001167-23,Nº,DET,O
2,EudraCT Nº : 2018-001167-23,:,PUNCT,O
3,EudraCT Nº : 2018-001167-23,2018-001167-23,NUM,O
4,Título público : Un estudio de Sacituzumab Gov...,Título,PROPN,O
...,...,...,...,...
34236,Nombre del paciente ( o testigo ) * : _ _ _ _ ...,_,ADJ,O
34237,Nombre del paciente ( o testigo ) * : _ _ _ _ ...,_,ADJ,O
34238,Nombre del paciente ( o testigo ) * : _ _ _ _ ...,_,ADJ,O
34239,Nombre del paciente ( o testigo ) * : _ _ _ _ ...,_,ADJ,O


In [33]:

df_dev_complete = df_dev_complete.replace([np.nan, -np.inf], 0)





# Generate features for the Dev set.

In [34]:
examples_dev = []
for index, group in df_dev_complete.groupby('Sentence'):
    words = list(group['Word'])
    pos = list(group['POS'])
    tags = list(group['Tag'])

    for index, word in enumerate(words):
        if index == 0:
            prevWord = '<start>'
        else:
            prevWord = words[index - 1]

        if index + 1 < len(words):
            nextWord = words[index + 1]
        else:
            nextWord = '<end>'
        examples_dev.append(GenerateFeaturesForSentence(word, prevWord, nextWord, pos[index], tags[index]))

In [35]:
examples_dev[0]
len(examples_dev)

34241

In [36]:
lista_all_dev=[]
tupla_dev=()
for tok_dev, tag_dev in zip (all_tokens_dev, all_tags_dev):
  tupla_dev= (tok_dev, tag_dev)
  lista_all_dev.append(tupla_dev)

In [37]:
print(len(lista_all_dev))
print(lista_all_dev[2])

1714
(['Título', 'científico', ':', 'Estudio', 'en', 'fase', 'II', 'abierto', 'de', 'sacituzumab', 'govitecán', 'en', 'el', 'cáncer', 'urotelial', 'metastásico', 'o', 'localmente', 'avanzado', 'irresecable'], ['O', 'O', 'O', 'B-CW', 'I-CW', 'I-CW', 'I-CW', 'I-CW', 'O', 'B-CW', 'I-CW', 'O', 'O', 'B-CW', 'I-CW', 'B-CW', 'O', 'B-CW', 'I-CW', 'B-CW'])


This [link](https://saturncloud.io/blog/how-to-create-new-values-in-a-pandas-dataframe-column-based-on-values-from-another-column/#:~:text=To%20create%20the%20new%20column,and%20returns%20the%20corresponding%20category.) is interesting to add columns based on conditions in a dataframe.

# Processing data in the TEST set

In [38]:
ruta_test = "/content/drive/MyDrive/data_Federico_Ortega/TEST.csv"
data_test = pd.read_csv(ruta_test)
data_test

,tokens,ner_tags
0,EudraCT Nº : 2018-002981-37,O O O O
1,Título público : Estudio de extensión abierto ...,O O O B-CW I-CW I-CW I-CW O B-CW O O O O O O O...
2,Título científico : Estudio de extensión abier...,O O O B-CW I-CW I-CW I-CW O B-CW O O O O O O O...
3,Indicación : Convulsiones Tónico-Clónicas Gene...,B-CW O B-CW I-CW I-CW I-CW
4,Criterios de inclusión :,O O O O
...,...,...
2092,"En este supuesto , ( cuando la unidad de SCU d...",O O O O O O O O O B-CW O O O O O O O O O O O O...
2093,"Para ello , junto con la solicitud , deberá pr...",O O O O O O O O O O O
2094,Certificación o documento que demuestre que el...,O O O O O O O O O B-CW O O O O O O O O O O O B...
2095,Certificación o documento que demuestre la exi...,O O O O O O O O O O O O O O B-CW O O O O O O O...


In [39]:
array_tags_test=data_test.loc[:,"ner_tags"].values
len(array_tags_test)
all_tags_test=[]
for tag_test in array_tags_test:
  lista_tag_test=tag_test.split()
  all_tags_test.append(lista_tag_test)


In [40]:
array_tokens_test=data_test.loc[:,"tokens"].values
len(array_tokens_test)
all_tokens_test=[]
for token_test in array_tokens_test:
  lista_token_test=token_test.split()
  all_tokens_test.append(lista_token_test)

In [41]:
lista_all_test=[]
tupla_test=()
for tok_test, tag_test in zip (all_tokens_test, all_tags_test):
  tupla_test= (tok_test, tag_test)
  lista_all_test.append(tupla_test)

# Adding POS from spaCy to the TEST set

In [43]:
# array tokens contains the tokens of the complex words
array_tokens_test=data_test.loc[:,"tokens"].values
print(len(array_tokens_test))
all_tokens_test=[]
all_tags_test=[]
all_pos_test=[]
tuple_sent_token_test=()
list_tuple_sent_token_test=[]
for token_test, tag_test in zip(array_tokens_test, array_tags_test):
  lista_pos_test=[]
  lista_token_test=token_test.split()
  lista_tag_test=tag_test.split()
  for t_test in lista_token_test:
    pos_test=nlp_es(t_test)
    pos_test=pos_test[0].pos_
    lista_pos_test.append(pos_test)
  all_tokens_test.append(lista_token_test)
  all_tags_test.append(lista_tag_test)
  all_pos_test.append(lista_pos_test)
  tuple_sent_token_test=(token_test, lista_token_test, lista_pos_test, lista_tag_test)
  list_tuple_sent_token_test.append(tuple_sent_token_test)


2097


In [44]:
list_sent_word_test=[]
tuple_sent_test=()
for element_t in list_tuple_sent_token_test:
  for word_t, pos_t, tag_t in zip(element_t[1],element_t[2], element_t[3]):
    tuple_sent_test=(element_t[0], word_t, pos_t, tag_t)

    list_sent_word_test.append(tuple_sent_test)


list_sent_word_test[0]

('EudraCT Nº : 2018-002981-37', 'EudraCT', 'NOUN', 'O')

In [45]:
df_test_complete = pd.DataFrame(list_sent_word_test, columns =['Sentence', 'Word', 'POS', 'Tag'])
df_test_complete

,Sentence,Word,POS,Tag
0,EudraCT Nº : 2018-002981-37,EudraCT,NOUN,O
1,EudraCT Nº : 2018-002981-37,Nº,DET,O
2,EudraCT Nº : 2018-002981-37,:,PUNCT,O
3,EudraCT Nº : 2018-002981-37,2018-002981-37,NUM,O
4,Título público : Estudio de extensión abierto ...,Título,PROPN,O
...,...,...,...,...
42635,Serologías de la madre realizadas en el contex...,el,DET,O
42636,Serologías de la madre realizadas en el contex...,RD,PROPN,B-CW
42637,Serologías de la madre realizadas en el contex...,Ley,PROPN,O
42638,Serologías de la madre realizadas en el contex...,9/2014,NOUN,O


In [46]:
df_dev_complete = df_dev_complete.replace([np.nan, -np.inf], 0)

# Generate features for the TEST set.

In [47]:
examples_test = []
for index, group in df_test_complete.groupby('Sentence'):
    words = list(group['Word'])
    pos = list(group['POS'])
    tags = list(group['Tag'])

    for index, word in enumerate(words):
        if index == 0:
            prevWord = '<start>'
        else:
            prevWord = words[index - 1]

        if index + 1 < len(words):
            nextWord = words[index + 1]
        else:
            nextWord = '<end>'
        examples_test.append(GenerateFeaturesForSentence(word, prevWord, nextWord, pos[index], tags[index]))

In [48]:
examples_dev[0]
len(examples_dev)

34241

# Model Training

### Prepare TRAIN and DEV sets

In [49]:
X_train_Orig, X_test, y_train_Orig, y_test = [i[0] for i in examples], [i[0] for i in examples_dev], [i[1] for i in examples],[i[1] for i in examples_dev]
#X_train_Orig, X_test, y_train, y_test = train_test_split(
#   [i[0] for i in examples], [i[1] for i in examples], test_size=0.33, random_state=0)

In [50]:
print(X_train_Orig[1000])
print(X_test[1000])
print(len(X_train_Orig),len(X_test))

{'lowercase': '.', 'prevword': 'hidrocefalia', 'nextword': '<end>', 'iscaps': 'False', 'istitlecase': 'False', 'isdigit': 'False', 'pos': 'PUNCT'}
{'lowercase': 'es', 'prevword': 'orina', 'nextword': 'débil', 'iscaps': 'False', 'istitlecase': 'False', 'isdigit': 'False', 'pos': 'AUX'}
38691 34241


In [51]:
print(y_train_Orig[1000])
print(y_test[1000])
print(len(y_train_Orig),len(y_test))

O
O
38691 34241


## Vectorizing text and tokens

In [52]:
vectorizer = DictVectorizer(sparse=False)
X_train = vectorizer.fit_transform(X_train_Orig)

In [53]:
print(X_train_Orig[0])
vector = vectorizer.transform(X_train_Orig[0])
print(vector)
print(len(vector[0]))
print(vectorizer.inverse_transform(vector))

print(X_train_Orig[1])
vector = vectorizer.transform(X_train_Orig[1])
print(vector)
print(vectorizer.inverse_transform(vector))

{'lowercase': '(', 'prevword': '<start>', 'nextword': '*', 'iscaps': 'False', 'istitlecase': 'False', 'isdigit': 'False', 'pos': 'PUNCT'}
[[1. 0. 1. ... 0. 0. 0.]]
14907
[{'iscaps=False': 1.0, 'isdigit=False': 1.0, 'istitlecase=False': 1.0, 'lowercase=(': 1.0, 'nextword=*': 1.0, 'pos=PUNCT': 1.0, 'prevword=<start>': 1.0}]
{'lowercase': '*', 'prevword': '(', 'nextword': 'Solo', 'iscaps': 'False', 'istitlecase': 'False', 'isdigit': 'False', 'pos': 'SYM'}
[[1. 0. 1. ... 0. 0. 0.]]
[{'iscaps=False': 1.0, 'isdigit=False': 1.0, 'istitlecase=False': 1.0, 'lowercase=*': 1.0, 'nextword=Solo': 1.0, 'pos=SYM': 1.0, 'prevword=(': 1.0}]


In [54]:
X_test = vectorizer.transform(X_test)
#y_train = vectorizer.transform(y_train_Orig)

In [55]:
print(X_train[0])
#print(y_train[0])


[1. 0. 1. ... 0. 0. 0.]


## **Logistic Regression**
## Training with word features in the dataframes `df_complete`and `df_complete_dev`

# Train the model and evaluate it against the DEV set

In [59]:
# Import LogisticRegression classifier
from sklearn.linear_model import LogisticRegression
# Import cross-validation library
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler

# Evaluation using 5-fold cross-validation on the DEV set

In [61]:
#clf = LogisticRegression(random_state=0, max_iter=50).fit(X_train, y_train_Orig)
# Use class_weight=’balanced’ for data with unbalanced classes
clf_LR= LogisticRegression(random_state=0, max_iter=200, solver='liblinear',class_weight='balanced').fit(X_train, y_train_Orig)
scores = cross_val_score(clf_LR, X_train, y_train_Orig, cv=5, scoring='f1_micro')

print(f"F1 scores for each fold: {scores}")
print(f"Mean F1 score: {np.mean(scores)}")



F1 scores for each fold: [0.90851531 0.89402946 0.89868183 0.85655208 0.86831223]
Mean F1 score: 0.8852181825972589


In [74]:
print('Train Classification Report: ')
pred_train = clf_LR.predict(X_train)
print(classification_report(y_train_Orig, pred_train))

Train Classification Report: 
              precision    recall  f1-score   support

        B-CW       0.85      0.95      0.90      5133
        I-CW       0.85      0.91      0.88      3767
           O       0.99      0.96      0.97     29791

    accuracy                           0.95     38691
   macro avg       0.90      0.94      0.92     38691
weighted avg       0.96      0.95      0.95     38691



# Results on the TEST set

In [75]:
print('Test Classification Report: ')
pred_test = clf_LR.predict(X_test)
print(classification_report(y_test, pred_test))

Test Classification Report: 
              precision    recall  f1-score   support

        B-CW       0.52      0.75      0.61      2836
        I-CW       0.20      0.62      0.30      2032
           O       0.96      0.77      0.86     29373

    accuracy                           0.76     34241
   macro avg       0.56      0.71      0.59     34241
weighted avg       0.88      0.76      0.80     34241



In [70]:
from sklearn.metrics import precision_recall_fscore_support
precision, recall, fscore, support = precision_recall_fscore_support(y_test, pred_test, average='micro')
print(f"Micro-average Precision: {precision}")
print(f"Micro-average Recall: {recall}")
print(f"Micro-average F1-Score: {fscore}")

Micro-average Precision: 0.7632954645016209
Micro-average Recall: 0.7632954645016209
Micro-average F1-Score: 0.7632954645016209


In [63]:
print('Testing Accuracy: ', clf_LR.score(X_test, y_test))

Testing Accuracy:  0.7632954645016209


## Save model as pickle file

In [77]:
import pickle
with open('LR_model_vectorizer.pkl', 'wb') as model_LR:
    pickle.dump((vectorizer, clf_LR), model_LR)

# SVM model

## Preparing the datasets from TRAIN and DEV

In [56]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X_train_Orig, X_test, y_train_Orig, y_test = [i[0] for i in examples], [i[0] for i in examples_dev], [i[1] for i in examples],[i[1] for i in examples_dev]



v1 = DictVectorizer(sparse=False)
X_train = v1.fit_transform(X_train_Orig)

X_test= v1.transform(X_test)


## Train and evaluate the model using the TRAIN and DEV sets

In [79]:
from sklearn import svm

In [80]:
svm_m = svm.SVC(random_state=0, max_iter=200, kernel='linear',class_weight='balanced').fit(X_train, y_train_Orig)
scores = cross_val_score(svm_m, X_train, y_train_Orig, cv=5, scoring='f1_micro')

print(f"F1 scores for each fold: {scores}")
print(f"Mean F1 score: {np.mean(scores)}")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-

F1 scores for each fold: [0.14898566 0.12302921 0.11553373 0.14034634 0.13130008]
Mean F1 score: 0.1318390026968968


## Evaluate the model on the TEST set

In [81]:
X_test, y_test = [i[0] for i in examples_test], [i[1] for i in examples_test]
X_test = v1.transform(X_test)

pred_test = svm_m.predict(X_test)
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

        B-CW       0.14      0.78      0.24      3126
        I-CW       0.05      0.58      0.09      2060
           O       0.98      0.00      0.00     37454

    accuracy                           0.09     42640
   macro avg       0.39      0.45      0.11     42640
weighted avg       0.87      0.09      0.02     42640



In [82]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test, pred_test, average='micro')
print(f"Micro-average Precision: {precision}")
print(f"Micro-average Recall: {recall}")
print(f"Micro-average F1-Score: {fscore}")

Micro-average Precision: 0.08574108818011257
Micro-average Recall: 0.08574108818011257
Micro-average F1-Score: 0.08574108818011257


## Save the model to pickle file

In [83]:
with open('SVC_model_vectorizer.pkl', 'wb') as model_SVM:
    pickle.dump((vectorizer, svm_m), model_SVM)

# LinearSVM

## Train and evaluate the model using the TRAIN and DEV sets


In [57]:
from sklearn.svm import LinearSVC

In [60]:
svm_linear = LinearSVC(random_state=0, max_iter=200, class_weight='balanced').fit(X_train, y_train_Orig)
scores = cross_val_score(svm_linear, X_train, y_train_Orig, cv=5, scoring='f1_micro')

print(f"F1 scores for each fold: {scores}")
print(f"Mean F1 score: {np.mean(scores)}")

F1 scores for each fold: [0.92337511 0.90436805 0.90566038 0.87089687 0.91044197]
Mean F1 score: 0.9029484782860413


## Evaluate the model on the TEST set

In [62]:
X_test, y_test = [i[0] for i in examples_test], [i[1] for i in examples_test]
X_test = v1.transform(X_test)

pred_test = svm_linear.predict(X_test)
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

        B-CW       0.52      0.68      0.59      3126
        I-CW       0.16      0.61      0.25      2060
           O       0.96      0.78      0.86     37454

    accuracy                           0.77     42640
   macro avg       0.55      0.69      0.57     42640
weighted avg       0.89      0.77      0.81     42640



In [64]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, fscore, support = precision_recall_fscore_support(y_test, pred_test, average='micro')
print(f"Micro-average Precision: {precision}")
print(f"Micro-average Recall: {recall}")
print(f"Micro-average F1-Score: {fscore}")

Micro-average Precision: 0.7660881801125704
Micro-average Recall: 0.7660881801125704
Micro-average F1-Score: 0.7660881801125704


In [68]:
print('Testing Accuracy: ', svm_linear.score(X_test, y_test))

Testing Accuracy:  0.7660881801125704


## Save the model to pickle file

In [67]:
import pickle
with open('LinearSVM_model_vectorizer.pkl', 'wb') as model_SVM:
    pickle.dump((vectorizer, svm_linear), model_SVM)